CSCI E-118 Introduction to Blockchain and Bitcoin
Solidity Coding Homework
Julian Avila

##  Problems Summary:

1) Write this function as a contract in Solidity.

2) Ammend the Solidity Voting contract where there are functions such that:
    
 - the inputs are the number of Voters, an integer
 - once a number gets more than 50% of the votes, they are stored as the winner
 - a function that asks who the winner is; if there is no winner yet, mention that there is no winner

De-couple functionality as much as possible. Do not write one function that will attempt every task above.

3) Use a Mapping (dictionary-like) structure in Solidity to cache/memorize factorial values, like we did in Python with FactorialMemo.

Additional context is included below:

In [ ]:
# Initialize the necessary files:

from solcx import set_solc_version,compile_source, compile_files, link_code

import json
import web3

from web3 import Web3, HTTPProvider
from web3.contract import ConciseContract


set_solc_version('v0.4.25')


def get_contract_interface(input_str, contract_source_code):
    # Solidity Compiler

    compiled_sol = compile_source(contract_source_code) # Compiled source code
    contract_interface = compiled_sol['<stdin>:' + input_str]

    # web3.py instance
    w3 = Web3(Web3.EthereumTesterProvider())

    # set pre-funded account as sender
    w3.eth.defaultAccount = w3.eth.accounts[0]

    # Instantiate and deploy contract
    ContractDeploy = w3.eth.contract(abi=contract_interface['abi'], bytecode=contract_interface['bin'])

    # Submit the transaction that deploys the contract
    tx_hash = ContractDeploy.constructor().transact()

    # Wait for the transaction to be mined, and get the transaction receipt
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

    # Create the contract instance with the newly-deployed address
    contract_inst = w3.eth.contract(
        address=tx_receipt.contractAddress,
        abi=contract_interface['abi'],
    )
    
    return w3, contract_inst

In [ ]:
# Below if AverageStream from Assignment 0:

# This class keeps tracks of an average without saving a list of numbers.

class AverageStream:
    
    def __init__(self):
        self.sum = 0 # float type
        self.number_numbers = 0 # integer type
        
        # self.sum = None
        # self.number_numbers = None
        
    def add_element(self, ele):
        self.sum += ele
        self.number_numbers += 1
        return self
    
    def get_average(self): # concerned with the inputs (self.sum, self.number_numbers)
        return (self.sum/self.number_numbers)
    
    def add_get_average(self, ele):
        return self.add_element(ele).get_average()

1) Write this function as a contract in Solidity.

In [26]:
contract_source_code = '''
pragma solidity ^0.4.25;

contract Voting {

  mapping (int => uint8) public votesReceived;

  int[5] candidateList = [int(0),1,2,3,4];
    

  

  function totalVotesFor(int candidate) view public returns (uint8) {

    require(validCandidate(candidate));

    return votesReceived[candidate];

  }



  function voteForCandidate(int candidate) public {

    require(validCandidate(candidate));

    votesReceived[candidate] += 1;

  }



  function validCandidate(int candidate) view public returns (bool) {

    for(uint i = 0; i < candidateList.length; i++) {

      if (candidateList[i] == candidate) {

        return true;

      }

    }

    return false;

  }
}
'''

input_str = 'Voting'
w3, contract_inst = get_contract_interface(input_str, contract_source_code)

In [27]:
# Voting 0 twice
tx_hash = contract_inst.functions.voteForCandidate(0).transact()
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

tx_hash = contract_inst.functions.voteForCandidate(0).transact()
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

# Vote for 1
tx_hash = contract_inst.functions.voteForCandidate(1).transact()
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

# Vote for 2
tx_hash = contract_inst.functions.voteForCandidate(2).transact()
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)


In [28]:
contract_inst.functions.totalVotesFor(0).call()

2

In [24]:
contract_inst.functions.totalVotesFor(2).call()

1

2) Ammend the Solidity Voting contract where there are functions such that:
    
 - the inputs are the number of Voters, an integer
 - once a number gets more than 50% of the votes, they are stored as the winner
 - a function that asks who the winner is; if there is no winner yet, mention that there is no winner

De-couple functionality as much as possible. Do not write one function that will attempt every task above.

In [13]:
# Recall from class the factorial function in Python.
def factorial(n):
    
    if n <= 1:
        return 1
    else:
        return n * factorial(n-1)

# Class that memorizes factorial values:
    
class FactorialMemo:

    def __init__(self):
        self.memo_dict = {0:1, 1:1}
        
    def factorial(self, n):
        if n in self.memo_dict:
            return self.memo_dict[n]
        self.memo_dict[n] = n * self.factorial(n- 1)
        return self.memo_dict[n]

factorial_memo = FactorialMemo()

factorial_memo.factorial(50)

30414093201713378043612608166064768844377641568960512000000000000

In [35]:
# ...and its Solidity counterpart:
contract_source_code = '''
pragma solidity ^0.4.25;

// mapping-based contract definition:

contract FactorialWrap {
    
    
    

    function factorial(uint n) returns (uint) 
    {
        
        if (n <= 1) 
        {   return 1; } 
        
        else {   return n * factorial(n - 1); }  
    }


}
'''
input_str = 'FactorialWrap'
w3, contract_inst = get_contract_interface(input_str, contract_source_code)


# Transactional form
tx_hash = contract_inst.functions.factorial(5).transact()
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
# Look at results form/ Testing form
assert(factorial(5)==contract_inst.functions.factorial(5).call())

3) Use a Mapping (dictionary-like) structure in Solidity to cache/memorize factorial values, like we did in Python with FactorialMemo.